In [4]:
# Module importing
import pandas as pd

#Dataset Path
DATASET_PATH = "./data/"

In [7]:
# Importing dataset

# NOTA: hay que hacer un pip install pyarrow para que funcione

df_feb= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-02.parquet', engine='pyarrow')
df_aug= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-08.parquet', engine='pyarrow')
df_nov= pd.read_parquet(DATASET_PATH+'yellow_tripdata_2020-11.parquet', engine='pyarrow')

# 1- Analisis exploratorio inicial

## 1.1 - Dataset features and characteristics

In [11]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-02-01 00:17:35,2020-02-01 00:30:32,1.0,2.60,1.0,N,145,7,1,11.0,0.5,0.5,2.45,0.0,0.3,14.75,0.0,None
1,1,2020-02-01 00:32:47,2020-02-01 01:05:36,1.0,4.80,1.0,N,45,61,1,21.5,3.0,0.5,6.30,0.0,0.3,31.60,2.5,None
2,1,2020-02-01 00:31:44,2020-02-01 00:43:28,1.0,3.20,1.0,N,186,140,1,11.0,3.0,0.5,1.00,0.0,0.3,15.80,2.5,None
3,2,2020-02-01 00:07:35,2020-02-01 00:31:39,1.0,4.38,1.0,N,144,140,1,18.0,0.5,0.5,3.00,0.0,0.3,24.80,2.5,None
4,2,2020-02-01 00:51:43,2020-02-01 01:01:29,1.0,2.28,1.0,N,238,152,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80,0.0,None


In [14]:
df_feb.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                      object
dtype: object

## 1.2 Handling missing data

In [20]:
df_feb.isna().values.any()

True

In [16]:
pd.isna(df_feb)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299362,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
6299363,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
6299364,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
6299365,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True
